In [14]:
import sqlite3 as sql
import pandas as pd

In [15]:
arq = pd.read_csv("TabelaAtividade.csv", delimiter=';')
arq.head(100)

,Código Pedido,Produto,Quant.,Valor Uni,Valor Total,Cliente,Entregue por,Data Pedido,Data Pagamento,Data Liberação,Data Envio,Data Receb.
0,1,Coca Cola,3,10,30,Victor,Gela Goela,29/06/2024 06:00,30/06/2024,30/06/2024 07:00,01/07/2024,NaN
1,1,Cerveja,5,34,153,Victor,Gela Goela,29/06/2024 10:00,30/06/2024,30/06/2024 09:00,30/06/2024,01/07/2024 10:00
2,1,Biscoito,4,5,20,Victor,Americanas,29/06/2024 15:00,30/06/2024,30/06/2024 10:00,NaN,NaN
3,2,Livro,1,45,45,Henrique,Amazon,30/06/2024 09:00,30/06/2024,NaN,NaN,NaN
4,2,Biscoito,2,5,10,Henrique,Americanas,30/06/2024 11:00,30/06/2024,NaN,NaN,NaN
5,2,Livro,1,60,60,Henrique,Amazon,30/06/2024 18:00,30/06/2024,NaN,NaN,NaN
6,3,Cerveja,2,34,68,José,Gela Goela,28/06/2024 11:00,28/06/2024,01/07/2024 09:00,01/07/2024,01/07/2024 15:00
7,4,Vinho,2,44,66,Maria,Bretas,28/06/2024 14:00,29/06/2024,30/06/2024 08:00,30/06/2024,01/07/2024 20:00
8,5,Cerveja,3,34,102,Victor,Gela Goela,28/06/2024 17:00,29/06/2024,30/06/2024 08:00,NaN,NaN


In [17]:
path = "data_warehouse.db"
connection = sql.connect(path)

# -----------------------------------------
# Transformação para a Tabela Fato Transacional
# -----------------------------------------
fato_transacional = arq[["Código Pedido", "Produto", "Quant.", "Valor Uni", "Valor Total", "Cliente", "Data Liberação"]]
fato_transacional = fato_transacional.dropna(subset=["Data Liberação"])  # Excluir registros sem liberação

# Carregar para o SQLite
fato_transacional.to_sql("fato_transacional", connection, if_exists="replace", index=False)

# -----------------------------------------
# Transformação para a Tabela Fato Periódico (Agregação diária)
# -----------------------------------------
fato_periodico = arq.dropna(subset=["Data Liberação"]) \
    .groupby(["Data Liberação", "Produto", "Cliente"]) \
    .agg(quantidade=("Quant.", "sum"), total_vendas=("Valor Total", "sum")) \
    .reset_index()

# Carregar para o SQLite
fato_periodico.to_sql("fato_periodico", connection, if_exists="replace", index=False)

# -----------------------------------------
# Transformação para a Tabela Fato Acumulativo
# -----------------------------------------
fato_acumulativo = arq.groupby(["Data Pedido", "Data Liberação"]) \
    .agg(total_pedido=("Valor Total", "sum"), total_venda=("Valor Total", "sum")) \
    .reset_index()

# Carregar para o SQLite
fato_acumulativo.to_sql("fato_acumulativo", connection, if_exists="replace", index=False)



print("ETL concluído com sucesso!")

ETL concluído com sucesso!


In [18]:
fato_acumulativo

,Data Pedido,Data Liberação,total_pedido,total_venda
0,28/06/2024 11:00,01/07/2024 09:00,68,68
1,28/06/2024 14:00,30/06/2024 08:00,66,66
2,28/06/2024 17:00,30/06/2024 08:00,102,102
3,29/06/2024 06:00,30/06/2024 07:00,30,30
4,29/06/2024 10:00,30/06/2024 09:00,153,153
5,29/06/2024 15:00,30/06/2024 10:00,20,20


In [19]:
fato_periodico

,Data Liberação,Produto,Cliente,quantidade,total_vendas
0,01/07/2024 09:00,Cerveja,José,2,68
1,30/06/2024 07:00,Coca Cola,Victor,3,30
2,30/06/2024 08:00,Cerveja,Victor,3,102
3,30/06/2024 08:00,Vinho,Maria,2,66
4,30/06/2024 09:00,Cerveja,Victor,5,153
5,30/06/2024 10:00,Biscoito,Victor,4,20


In [20]:
fato_transacional

,Código Pedido,Produto,Quant.,Valor Uni,Valor Total,Cliente,Data Liberação
0,1,Coca Cola,3,10,30,Victor,30/06/2024 07:00
1,1,Cerveja,5,34,153,Victor,30/06/2024 09:00
2,1,Biscoito,4,5,20,Victor,30/06/2024 10:00
6,3,Cerveja,2,34,68,José,01/07/2024 09:00
7,4,Vinho,2,44,66,Maria,30/06/2024 08:00
8,5,Cerveja,3,34,102,Victor,30/06/2024 08:00


In [25]:
table = pd.read_sql_query("SELECT * FROM fato_transacional WHERE CLIENTE <> 'Victor'",connection)
table

,Código Pedido,Produto,Quant.,Valor Uni,Valor Total,Cliente,Data Liberação
0,3,Cerveja,2,34,68,José,01/07/2024 09:00
1,4,Vinho,2,44,66,Maria,30/06/2024 08:00


In [ ]:
# Fechar conexão
connection.close()